In [4]:
from icecube import dataio, dataclasses, simclasses, phys_services
import os, sys
import numpy as np

In [7]:
# Let's open a file first
mc_dir = '/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999'
file_list = os.listdir(mc_dir)
infile_name = os.path.join(mc_dir, file_list[0])
print(infile_name)

/data/icecube/domeff_analysis/reco_sim_nominal/0000000-0000999/83.i3.bz2


In [8]:
infile = dataio.I3File(infile_name)
frame = infile.pop_physics()
print frame

[ I3Frame  (Physics):
  'AnalysisMuons' [Physics] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (174)
  'BackgroundCLSim_intermediatePhotons' [DAQ] ==> I3Map<ModuleKey, I3Vector<I3Photon> > (3167)
  'BackgroundI3MCPESeriesMap_0.990' [DAQ] ==> I3Map<OMKey, vector<I3MCPE> > (90)
  'BackgroundI3MCTree' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (1786)
  'BackgroundI3MCTree_preMuonProp' [DAQ] ==> TreeBase::Tree<I3Particle, I3ParticleID, __gnu_cxx::hash<I3ParticleID> > (918)
  'BackgroundMMCTrackList' [DAQ] ==> I3Vector<I3MMCTrack> (304)
  'BeaconLaunches' [DAQ] ==> I3Map<OMKey, vector<I3DOMLaunch> > (46)
  'CalibratedWaveformRange' [DAQ] ==> I3TimeWindow (48)
  'CascadeFilter_13' [Physics] ==> I3PODHolder<bool> (27)
  'CorsikaInteractionHeight' [DAQ] ==> I3PODHolder<double> (36)
  'CorsikaWeightMap' [DAQ] ==> I3Map<string, double> (482)
  'DCAnalysisHits' [Physics] ==> I3PODHolder<double> (36)
  'DCNHits' [Physics] ==> 

In [10]:
track_list = frame['MMCTrackList']
mctree     = frame['I3MCTree']

In [11]:
# Let's find the muon
print(track_list)

[I3MMCTrack = [
 (xi, yi, zi, ti, Ei) = (-115.176 ,542.973 ,800 ,7702.91 ,559.519)
 (xc, yc, zc, tc, Ec) = (-29.4748 ,181.081 ,-77.554 ,10882.1 ,267.507)
 (xf, yf, zf, tf, Ef) = (41.079 ,-116.846 ,-800 ,13499.4 ,60.5039)
 Elost = 499.015
 Particle = [ I3Particle MajorID : 13469127016857373990
             MinorID : 846
              Zenith : 0.400847
             Azimuth : 1.80333
                   X : -227.484
                   Y : 1017.22
                   Z : 1949.99
                Time : 3536.68
              Energy : 1054.01
               Speed : 0.299792
              Length : 3125.59
                Type : MuPlus
        PDG encoding : -13
               Shape : StartingTrack
              Status : NotSet
            Location : InIce
]]
]


In [12]:
# There's only one track in the list, so it is easy
# Note that in some cases there will be multiple tracks in this list
# If both tracks enter the smaller volume (500m one) we want to mark those events as a "bundle"
# Bundles are bad. We don't want them, but we will cover that later. For now, you can restrict yourself to single muons

In [37]:
# Let's do the energy accounting exercise
# Pick the muon track at its injection point and the length it traveled
muon_track = track_list[0]
print('Injection point of the muon', muon_track.particle.pos)
print('Length that the muon traveled in total', muon_track.particle.length)

# Let's calculate the distance the muon traveled *inside the 800m volume*
entering_large_cylinder = dataclasses.I3Position(muon_track.xi, muon_track.yi, muon_track.zi)
print('Point where it enters the 800m volume', entering_large_cylinder)
d_injection_cylinder = (muon_track.particle.pos - entering_large_cylinder).magnitude
print('Distance from injection to 800m volume', d_injection_cylinder)
d_inside = muon_track.particle.length - d_injection_cylinder

# Ionization energy loss
econstant = 1/4.5 # GeV/m
e_ion = econstant*d_inside
print('Ionization losses', e_ion)

# Sum all the stochastics
daughters = mctree.get_daughters(muon_track.particle)
e_stochastic = 0
for d in daughters:
    e_stochastic += d.energy

print('Stochastic losses', e_stochastic)
print('Total E lost inside volume', e_stochastic+e_ion)
print('Energy available upon entering volume', muon_track.Ei)
print('This does not add up very nicely, why?')

('Injection point of the muon', I3Position(-227.484,1017.22,1949.99))
('Length that the muon traveled in total', 3125.5943701684264)
('Point where it enters the 800m volume', I3Position(-115.176,542.973,800))
('Distance from injection to 800m volume', 1249.0023827390578)
('Ionization losses', 417.0204416509708)
('Stochastic losses', 102.0584694051989)
('Total E lost inside volume', 519.0789110561697)
('Energy available upon entering volume', 559.5189264756167)
This does not add up very nicely, why?


In [38]:
print('This is a good special example. This muon actually left the volume. It did not decay inside.')
# This function can be used to move along a track
# I'm taking the muon, and moving along a track a distance equal to its length
# Then I'm printing the end point
print(muon_track.particle.shift_along_track(muon_track.particle.length))
print('That end point is outside the detector')
print('This muon entered and left. That is fine, but our energy accounting will not work as nicely')
print('There are probably some energy losses outside as it left, that we did not account for')
print('Still, we only care about stochastic energy losses INSIDE the 500m volume, so the fact the muon leaves is fine')

This is a good special example. This muon actually left the volume. It did not decay inside.
I3Position(53.5633,-169.563,-927.836)
That end point is outside the detector
This muon entered and left. That is fine, but our energy accounting will not work as nicely
There are probably some energy losses outside as it left, that we did not account for
Still, we only care about stochastic energy losses INSIDE the 500m volume, so the fact the muon leaves is fine


In [39]:
# Let's do it again for anothermuon

In [50]:
frame = infile.pop_physics()

In [51]:
track_list = frame['MMCTrackList']
mctree     = frame['I3MCTree']

In [52]:
muon_track = track_list[0]
# Check where it decayed
print(muon_track.particle.shift_along_track(muon_track.particle.length))


I3Position(-114.322,151.015,-250.738)
